In [1]:
import torchvision
import torch

model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)

In [2]:
model.load_state_dict(torch.load('best_steering_model_xy.pth'))

<All keys matched successfully>

In [3]:
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()

image_widget = ipywidgets.Image()

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [6]:
from jetbot import Robot

robot = Robot()

In [7]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [8]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [9]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']

    # Model -> normalized target point (x: left/right, y: forward)
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0   # verify this sign; try removing the 0.5 or flipping sign if needed

    x_slider.value = x
    y_slider.value = y

    base_speed = speed_gain_slider.value

    # Heading error; flip x if your camera/model is mirrored
    angle_new = np.arctan2(x, y)               # try np.arctan2(-x, y) if steering goes the wrong way

    # Unwrap and derivative (frame-based)
    d = angle_new - angle_last
    d = (d + np.pi) % (2*np.pi) - np.pi        # keep in [-π, π]
    angle = angle_new

    p = angle * steering_gain_slider.value
    dterm = d * steering_dgain_slider.value
    steer = p + dterm + 0.0 * steering_bias_slider.value  # keep bias 0 while testing

    # Mix in symmetric domain [-1, 1]
    left_cmd  = np.clip(base_speed + steer, -1.0, 1.0)
    right_cmd = np.clip(base_speed - steer, -1.0, 1.0)

    # Send directly; JetBot supports [-1, 1]
    robot.left_motor.value  = float(left_cmd)
    robot.right_motor.value = float(right_cmd)

    steering_slider.value = steer
    speed_slider.value = base_speed

    angle_last = angle

execute({'new': camera.value})


In [10]:
camera.observe(execute, names='value')

In [11]:
import time

camera.unobserve(execute, names='value')

time.sleep(10)  # add a small sleep to make sure frames have finished processing

robot.stop()

In [12]:
camera.stop()